In [4]:
# Install necessary libraries
!pip install flask flask_cors pyngrok pandas

from flask import Flask, request, jsonify
from pyngrok import ngrok
from flask_cors import CORS
import pandas as pd

# Define the recommendation function
def recommend_courses_based_on_skills(data, current_course):
    """
    Recommends courses based on skill comparison and difficulty level.
    """
    difficulty_mapping = {
        'beginner': 0,
        'intermediate': 1,
        'advanced': 2,
        'not calibrated': -1
    }

    data['Difficulty Level Numeric'] = data['Difficulty Level'].str.lower().map(difficulty_mapping)

    try:
        current_course_data = data.loc[data['Course Name'] == current_course]
        current_difficulty = current_course_data['Difficulty Level Numeric'].iloc[0]
        current_skills = current_course_data['Skills'].iloc[0]

        if current_difficulty == -1:
            return f"Course '{current_course}' is not calibrated for difficulty."
        if current_skills is None or current_skills.strip() == '':
            current_skills = []
        else:
            current_skills = [skill.strip().lower() for skill in current_skills.split()]

    except IndexError:
        return f"Error: Course '{current_course}' not found in the dataset."

    if current_difficulty == 2:
        return "Your current course is at an advanced level. It's time to develop new skills for further progression!"

    higher_level_courses = data[data['Difficulty Level Numeric'] == current_difficulty + 1]

    if higher_level_courses.empty:
        return "No courses are available one level higher in difficulty."

    recommended_courses = []
    for _, course_row in higher_level_courses.iterrows():
        course_name = course_row['Course Name']
        course_skills = course_row['Skills']

        if course_skills is None or course_skills.strip() == '':
            course_skills = []
        else:
            course_skills = [skill.strip().lower() for skill in course_skills.split()]

        skill_overlap = set(current_skills) & set(course_skills)
        if skill_overlap:
            recommended_courses.append((course_name, len(skill_overlap)))

    if not recommended_courses:
        return "No suitable courses found one level higher based on your skills."

    recommended_courses = sorted(recommended_courses, key=lambda x: x[1], reverse=True)[:5]
    return [course[0] for course in recommended_courses]

# Load dataset
file_path = '/content/Coursera.csv'  # Update path if necessary
data = pd.read_csv(file_path)

# Flask app setup
app = Flask(__name__)
CORS(app)

@app.route('/recommend', methods=['POST'])
def recommend():
    """Endpoint to recommend courses."""
    try:
        req_data = request.get_json()
        current_course = req_data['current_course']

        recommendations = recommend_courses_based_on_skills(data, current_course)
        if isinstance(recommendations, list):
            details = []
            for course in recommendations:
                course_details = data[data['Course Name'] == course].iloc[0]
                details.append({
                    "Course Name": course,
                    "Description": course_details['Course Description'],
                    "Difficulty Level": course_details['Difficulty Level']
                })
            return jsonify({
                "recommendations": details
            })
        else:
            return jsonify({"message": recommendations})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
    # Start Ngrok tunnel
    ngrok.set_auth_token('2rA9QT8w4GXftuLHB7rLNG5R8vM_6NbtYDUYp55c4bLwthA79')  # Replace with your Ngrok auth token
    public_url = ngrok.connect(5000)
    print(f"Server is live at {public_url}")
    app.run(port=5000)


Server is live at NgrokTunnel: "https://da0d-34-48-139-52.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [12/Jan/2025 13:18:56] "POST /recommend HTTP/1.1" 200 -
